In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Rice Grain Classification Model Training Notebook\n",
    "## Rice Grainpalette - Deep Learning\n",
    "\n",
    "This notebook contains the complete training pipeline for the rice grain classification model."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Dependencies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import necessary libraries\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from PIL import Image\n",
    "import os\n",
    "import random\n",
    "from tqdm import tqdm\n",
    "\n",
    "# Deep learning libraries\n",
    "import tensorflow as tf\n",
    "from tensorflow.keras import layers, models\n",
    "from tensorflow.keras.preprocessing.image import ImageDataGenerator\n",
    "from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau\n",
    "\n",
    "# Model evaluation\n",
    "from sklearn.metrics import classification_report, confusion_matrix\n",
    "\n",
    "# Set random seed for reproducibility\n",
    "seed = 42\n",
    "tf.random.set_seed(seed)\n",
    "np.random.seed(seed)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Preparation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define paths and parameters\n",
    "DATA_DIR = 'data/Rice_Image_Dataset/'\n",
    "IMAGE_SIZE = (224, 224)\n",
    "BATCH_SIZE = 32\n",
    "\n",
    "# List of rice varieties (adapt based on your dataset)\n",
    "RICE_VARIETIES = ['Basmati', 'Jasmine', 'Arborio', 'Brown', 'Wild']"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Data augmentation and preprocessing\n",
    "train_datagen = ImageDataGenerator(\n",
    "    rescale=1./255,\n",
    "    rotation_range=20,\n",
    "    width_shift_range=0.2,\n",
    "    height_shift_range=0.2,\n",
    "    shear_range=0.2,\n",
    "    zoom_range=0.2,\n",
    "    horizontal_flip=True,\n",
    "    fill_mode='nearest',\n",
    "    validation_split=0.2\n",
    ")\n",
    "\n",
    "# Load and augment training data\n",
    "train_generator = train_datagen.flow_from_directory(\n",
    "    DATA_DIR,\n",
    "    target_size=IMAGE_SIZE,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    class_mode='categorical',\n",
    "    subset='training',\n",
    "    shuffle=True\n",
    ")\n",
    "\n",
    "# Load validation data\n",
    "valid_generator = train_datagen.flow_from_directory(\n",
    "    DATA_DIR,\n",
    "    target_size=IMAGE_SIZE,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    class_mode='categorical',\n",
    "    subset='validation',\n",
    "    shuffle=False\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Model Architecture"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_model(input_shape, num_classes):\n",
    "    \"\"\"\n",
    "    Create a CNN model for rice grain classification\n",
    "    \"\"\"\n",
    "    base_model = tf.keras.applications.EfficientNetB0(\n",
    "        include_top=False,\n",
    "        weights='imagenet',\n",
    "        input_shape=input_shape,\n",
    "        pooling='avg'\n",
    "    )\n",
    "    \n",
    "    # Freeze the base model\n",
    "    base_model.trainable = False\n",
    "    \n",
    "    # Create new model on top\n",
    "    inputs = tf.keras.Input(shape=input_shape)\n",
    "    x = base_model(inputs, training=False)\n",
    "    x = tf.keras.layers.Dense(256, activation='relu')(x)\n",
    "    x = tf.keras.layers.Dropout(0.5)(x)\n",
    "    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)\n",
    "    \n",
    "    model = tf.keras.Model(inputs, outputs)\n",
    "    \n",
    "    return model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize model\n",
    "input_shape = IMAGE_SIZE + (3,)\n",
    "num_classes = len(RICE_VARIETIES)\n",
    "model = create_model(input_shape, num_classes)\n",
    "\n",
    "# Model summary\n",
    "model.summary()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Model Compilation and Training"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compile the model\n",
    "model.compile(\n",
    "    optimizer='adam',\n",
    "    loss='categorical_crossentropy',\n",
    "    metrics=['accuracy']\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Callbacks\n",
    "callbacks = [\n",
    "    ModelCheckpoint(\n",
    "        'models/rice_classifier_best.h5',\n",
    "        monitor='val_accuracy',\n",
    "        save_best_only=True,\n",
    "        mode='max',\n",
    "        verbose=1\n",
    "    ),\n",
    "    EarlyStopping(\n",
    "        monitor='val_loss',\n",
    "        patience=10,\n",
    "        restore_best_weights=True\n",
    "    ),\n",
    "    ReduceLROnPlateau(\n",
    "        monitor='val_loss',\n",
    "        factor=0.2,\n",
    "        patience=5,\n",
    "        min_lr=1e-6\n",
    "    )\n",
    "]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Train the model\n",
    "history = model.fit(\n",
    "    train_generator,\n",
    "    epochs=50,\n",
    "    validation_data=valid_generator,\n",
    "    callbacks=callbacks,\n",
    "    verbose=1\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Model Evaluation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot training history\n",
    "def plot_history(history):\n",
    "    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))\n",
    "    \n",
    "    # Plot accuracy\n",
    "    ax1.plot(history.history['accuracy'], label='Train Accuracy')\n",
    "    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')\n",
    "    ax1.set_title('Model Accuracy')\n",
    "    ax1.set_ylabel('Accuracy')\n",
    "    ax1.set_xlabel('Epoch')\n",
    "    ax1.legend()\n",
    "    \n",
    "    # Plot loss\n",
    "    ax2.plot(history.history['loss'], label='Train Loss')\n",
    "    ax2.plot(history.history['val_loss'], label='Validation Loss')\n",
    "    ax2.set_title('Model Loss')\n",
    "    ax2.set_ylabel('Loss')\n",
    "    ax2.set_xlabel('Epoch')\n",
    "    ax2.legend()\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "plot_history(history)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate on validation set\n",
    "y_pred = model.predict(valid_generator)\n",
    "y_pred_classes = np.argmax(y_pred, axis=1)\n",
    "y_true = valid_generator.classes\n",
    "\n",
    "# Classification report\n",
    "print(\"\\nClassification Report:\")\n",
    "print(classification_report(y_true, y_pred_classes, target_names=RICE_VARIETIES))\n",
    "\n",
    "# Confusion matrix\n",
    "plt.figure(figsize=(8, 6))\n",
    "cm = confusion_matrix(y_true, y_pred_classes)\n",
    "sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', \n",
    "            xticklabels=RICE_VARIETIES, \n",
    "            yticklabels=RICE_VARIETIES)\n",
    "plt.title('Confusion Matrix')\n",
    "plt.xlabel('Predicted')\n",
    "plt.ylabel('Actual')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Model Fine-tuning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Unfreeze some layers for fine-tuning\n",
    "base_model.trainable = True\n",
    "for layer in base_model.layers[:100]:\n",
    "    layer.trainable = False\n",
    "    \n",
    "# Recompile with lower learning rate\n",
    "model.compile(\n",
    "    optimizer=tf.keras.optimizers.Adam(1e-5),\n",
    "    loss='categorical_crossentropy',\n",
    "    metrics=['accuracy']\n",
    ")\n",
    "\n",
    "# Fine-tune for a few more epochs\n",
    "history_fine = model.fit(\n",
    "    train_generator,\n",
    "    epochs=10,\n",
    "    initial_epoch=history.epoch[-1],\n",
    "    validation_data=valid_generator,\n",
    "    callbacks=callbacks\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Save Final Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save the final model\n",
    "model.save('models/rice_classifier_final.h5')\n",
    "\n",
    "# Save the model in TensorFlow.js format for web deployment\n",
    "!pip install tensorflowjs\n",
    "!mkdir -p models/tfjs_model\n",
    "!tensorflowjs_converter --input_format=keras models/rice_classifier_final.h5 models/tfjs_model/"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Test Inference"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Function to make predictions on new images\n",
    "def predict_rice_variety(image_path, model):\n",
    "    \"\"\"\n",
    "    Predict the rice variety from an image\n",
    "    \"\"\"\n",
    "    # Load and preprocess image\n",
    "    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)\n",
    "    img_array = tf.keras.preprocessing.image.img_to_array(img)\n",
    "    img_array = img_array / 255.0\n",
    "    img_array = tf.expand_dims(img_array, axis=0)\n",
    "    \n",
    "    # Make prediction\n",
    "    predictions = model.predict(img_array)\n",
    "    predicted_class = RICE_VARIETIES[np.argmax(predictions)]\n",
    "    confidence = np.max(predictions) * 100\n",
    "    \n",
    "    return predicted_class, confidence\n",
    "\n",
    "# Test with a sample image (replace with your test image path)\n",
    "test_image_path = 'test_samples/sample_rice.jpg'\n",
    "variety, confidence = predict_rice_variety(test_image_path, model)\n",
    "print(f\"Predicted Rice Variety: {variety} (Confidence: {confidence:.2f}%)\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
